In [1]:
from utils import load_model
from data import build_corpus
import time
import re
import numpy as np
import drug_classifier as dc

In [2]:
hmm_model = load_model("./ckpts/hmm.pkl")

In [4]:
train_word_lists, train_tag_lists, word2id, tag2id = \
        build_corpus("train", data_dir="./Drug")

In [7]:
with open('corpus.txt', 'r', encoding='utf-8') as file:
    corpus = file.read().splitlines()

In [8]:
with open('vocabulary.txt', 'r', encoding='utf-8') as file:
    keys = file.read().splitlines()
med_re = '|'.join(keys)

In [ ]:
oov=[]
expected, actual = 0, 0
for line in corpus: 
    txts = re.split('。|\s+', line)
    for t in txts:
        expected += len(re.findall(med_re, t))
        try:
            entities = hmm_model.get_entities(t, word2id,tag2id,omask=0)
        except:
            print('Error:', t)
            continue
        actual += len(entities)
        for e in entities:
            if e[0] not in keys:
                print('Found new:',e, 'from', t)
                actual -= 1
                oov.append(e[0])

In [60]:
oov = np.unique(oov)
len(oov)

788

In [61]:
oov2 = []
for word in oov:
    if re.search('^\d', word):
        word = re.sub('^\d\.?', '', word)
    if re.search('、|，', word):
        oov2 += re.split('、|，',word)
    else:
        oov2.append(word)

In [62]:
oov2 = np.unique(oov2)

In [63]:
len(oov2)

762

In [53]:
c = dc.load('trained_drug_classifier')

In [64]:
filtered = []
for word in oov2:
    #print(word)
    if c.is_drug(word):
        filtered.append(word)

In [65]:
len(filtered)

647

In [7]:
loc = 7674
sent = train_word_lists[loc]
start = time.time()
pred = hmm_model.test([sent])
print(time.time()-start)
for i in range(len(sent)):
    print(sent[i], pred[0][i], train_tag_lists[loc][i])

0.008941650390625
卡 B-DRUG B-DRUG
茚 I-DRUG I-DRUG
西 I-DRUG I-DRUG
林 I-DRUG I-DRUG
（ O O
C O O
a O O
r O O
i O O
n O O
d O O
a O O
c O O
i O O
l O O
l O O
i O O
n O O
） O O
， O O
商 O O
品 O O
名 O O
卡 O O
丹 O O
青 O O
霉 O O
素 O O
、 O O
茚 O O
满 O O
酯 O O
羧 O O
苄 O O
青 O O
霉 O O
素 O O
、 O O
羧 O O
苄 O O
青 O O
霉 O O
素 O O
茚 O O
满 O O
酯 O O
、 O O
羧 O O
苄 O O
青 O O
霉 O O
素 O O
茚 O O
满 O O
酯 O O
钠 O O
、 O O
羧 O O
茚 O O
青 B-DRUG B-DRUG
霉 I-DRUG I-DRUG
素 I-DRUG I-DRUG
钠 I-DRUG I-DRUG
、 O O
羧 O O
茚 O O
苄 O O
青 O O
霉 O O
素 O O
、 O O
羧 O O
茚 O O
苄 O O
青 B-DRUG B-DRUG
霉 I-DRUG I-DRUG
素 I-DRUG I-DRUG
钠 I-DRUG I-DRUG
